# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

Note, the argo trigger is currently not working for this flow, so I'm manually triggering the flow.

In [2]:
!python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion argo-workflows create
!python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying fullstack.prod.champion.taxifareprediction to Argo Workflows...
It seems this is the first time you are deploying fullstack.prod.champion.taxifareprediction to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-cqkixzvdsy3tjqdh-0-ndkt
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-cqkixzvdsy3tjqdh-0-ndkt")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-cqkixzvdsy3tjqdh-0-ndkt
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping 

In [ ]:
!python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion argo-workflows create
!python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion argo-workflows trigger

In [ ]:
champ_namespace='production:mfprj-cqkixzvdsy3tjqdh-0-ndkt'

## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

I took the flow from week 3, which already included linear regression and some additional steps to weed out bad data.  Then I added one more predictor to the model, "passenger_count".

In [32]:
!python ../flows/cloud/taxi_fare_challenger.py --environment=conda --production --branch challenger argo-workflows create


Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying fullstack.prod.challenger.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-uj55bdalmyvezuhl-0-bixe
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-uj55bdalmyvezuhl-0-bixe")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-uj55bdalmyvezuhl-0-bixe
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
Workflow fullstack.prod.challenger.taxifareprediction for flow TaxiFarePredicti

In [33]:
!python ../flows/cloud/taxi_fare_challenger.py --environment=conda --production --branch challenger argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow fullstack.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-fullstack.prod.challenger.taxifareprediction-flq6b).
See the run in the UI at https://ui-pw-1851997417.outerbounds.dev/TaxiFarePrediction/argo-fullstack.prod.challenger.taxifareprediction-flq6b


In [7]:
challenger_namespace = 'production:mfprj-uj55bdalmyvezuhl-0-bixe'

In [31]:
from metaflow import namespace, Metaflow
namespace(challenger_namespace)
Metaflow().flows

[Flow('TaxiFarePrediction')]

## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [12]:
from metaflow import namespace, Metaflow, Flow

In [35]:
best_score = -1; winner = None; winner_namespace = None
for n,model in zip([champ_namespace, challenger_namespace],['champion','challenger']):
    namespace(n)
    run = Flow('TaxiFarePrediction').latest_successful_run
    acc_score = run.data.scores.mean()
    if acc_score > best_score:
        best_score = acc_score
        winner = model
        winner_namespace = n
print(f'The winner is {model} with accuracy {best_score} in namespace {winner_namespace}')

The winner is challenger with accuracy 0.8963054466599946 in namespace production:mfprj-uj55bdalmyvezuhl-0-bixe


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!